## GNNLens Demo

### Introduction
This code is aiming to reproduce the data used in the demo: https://gnnlens.github.io/

The data will be generated in the folder "examples/examples_data".

After generating the data, you can use the command line to start the backend and frontend and check them in the browser.

### Steps

1. Prepare environments for running this notebook.

`cd examples`

`conda env create -f environment.yml`

`cd ..`

`conda activate gnnlens`

`conda install dgl-cuda10.1==0.7.1 -c dglteam`

`cd server`

`pip install -r requirements.txt`

`python setup.py install`

`cd ..`

2. Unzip the rawdata.zip in "examples/rawdata" folder.

`cd examples`

`unzip rawdata.zip`

3. Run the whole jupyter notebook "examples/prepare_examples_data.ipynb" to generate data in the folder "examples/examples_data".

4. Start the GNNLens backend. A complied version of frontend is located in "server/gnnlens/visbuild".

`cd examples`

`gnnlens --port 6800 --logdir examples_data`

5. Check the GNNLens at browser.

http://localhost:6800


## Import Packages

In [1]:
import gnnlens.GNNLensWriter as GNNLensWriter
import time
from utils import load_json, load_cora_ml_data, convert_data_package_to_dgl_graph, infer_pipeline, \
                    calculate_subgraph, initialize_cora_ml_explainer, GNNExplainer_explain, \
                    infer_pipeline_2, calculate_subgraph_2, initialize_photo_explainer, load_photo_data, GetModel

Using backend: pytorch


## Global Config

In [2]:
### Dump Folder Name
dump_folder_name = "examples_data"

### Raw Data Config
root_path = "./rawdata"
data_path = root_path+'/data/'
model_config_path = root_path + "/models/model_config.json"


## Initialize

In [3]:
writer = GNNLensWriter(dump_folder_name)
models_list = load_json(model_config_path)

## Cora_ML data preprocessing

Add Graph into Writer

In [4]:
### Long time required.
basic_config = models_list["25"]["graph_config"]
data_package, graph, labels, num_classes, features, idx_train, graph_additional_info = load_cora_ml_data(data_path, basic_config)
dgl_graph = convert_data_package_to_dgl_graph(data_package)
writer.add_graph("Cora_ML", dgl_graph, labels, num_classes, features, calculate_metrics=True, additional_info=graph_additional_info)

> Calculating SPD metrics for Cora_ML
Finished: 0.006314992904663086
> Calculating KFS metrics for Cora_ML
Finished: 0.010930776596069336
> Calculating layout for Cora_ML
Finished: 0.0015311241149902344
Finish calculating metrics for Cora_ML.


Add Model Inference Results into Writer

In [5]:
predicted_classes_GAT, output_vector_GAT = infer_pipeline(models_list, "25", root_path, data_path, "cora_ml")
predicted_classes_GATWUF, output_vector_GATWUF = infer_pipeline(models_list, "26", root_path, data_path, "cora_ml")
predicted_classes_MLP, output_vector_MLP = infer_pipeline(models_list, "27", root_path, data_path, "cora_ml")
## Order: GAT --> MLP --> GATWUF
#"id":26, "name": "Cora_ML_GAT_Identity_Features", --> GATWUF
#"id":27  "name": "Cora_ML_GAT_Identity_Structure",  --> MLP
writer.add_model("Cora_ML", "GAT", predicted_classes_GAT, output_vector_GAT)
writer.add_model("Cora_ML", "MLP", predicted_classes_MLP, output_vector_MLP)
writer.add_model("Cora_ML", "GATWUF", predicted_classes_GATWUF, output_vector_GATWUF)


0.8615658362989324
0.802491103202847
0.7209964412811388


Add Explanation Results by Integrated Gradient into Writer

In [6]:
### Long time required.
net = GetModel(root_path, models_list, "25")['net']
## For Cora_ML, calculating two nodes 0, and 2010
## For Photo, calculating two nodes 0, and 115
calculate_subgraph(writer, net, data_package, 0, predicted_classes_GAT, "Cora_ML", dgl_graph)
calculate_subgraph(writer, net, data_package, 2010, predicted_classes_GAT, "Cora_ML", dgl_graph)

Attribution Time:  9.76262617111206
Attribution Time:  9.665288925170898


Add Explanation Results by GNNExplainer into Writer

In [7]:
prog_args, explainer = initialize_cora_ml_explainer(net, data_package, labels, graph, features, idx_train)
for explain_node_idx in [0, 2010]:
    print("Explain {}".format(explain_node_idx))
    start_time = time.time()
    GNNExplainer_explain(dgl_graph, explainer, prog_args, explain_node_idx, writer, "Cora_ML", features, 75)
    print("Time: {}".format(time.time() - start_time))

(1, 2810, 2810) (1, 2810, 2879) (1, 2810) (1, 280) (1, 2810, 7)
Namespace(align_steps=1000, batch_size=20, bias=True, bmname=None, bn=False, ckptdir='ckpt', clean_log=False, clip=2.0, cuda='0', dataset='syn1', dropout=0.0, explain_node=None, explainer_suffix='', gpu=False, graph_idx=-1, graph_mode=False, hidden_dim=20, logdir='log', lr=0.1, mask_act='sigmoid', mask_bias=False, method='base', multigraph_class=-1, multinode_class=-1, name_suffix='', num_epochs=100, num_gc_layers=2, opt='adam', opt_decay_rate=None, opt_decay_step=None, opt_restart=None, opt_scheduler='none', output_dim=20, pkl_fname=None, writer=False)
Explain 0
node label:  0
neigh graph idx:  0 0
Node predicted label:  0
ExplainModule(
  (model): WrapUpGATModel(
    (net): tgGAT(
      (attention_0): GATConv(2879, 8, heads=1)
      (attention_1): GATConv(2879, 8, heads=1)
      (attention_2): GATConv(2879, 8, heads=1)
      (attention_3): GATConv(2879, 8, heads=1)
      (attention_4): GATConv(2879, 8, heads=1)
      (at

## Photo data preprocessing

Add Graph into Writer

In [8]:
basic_config = models_list["20"]["graph_config"]
data_package_2, graph, labels, num_classes, features, L, graph_additional_info = load_photo_data(data_path, basic_config)
dgl_graph = convert_data_package_to_dgl_graph(data_package_2)
writer.add_graph("Photo", dgl_graph, labels, num_classes, features, calculate_metrics=True, additional_info=graph_additional_info)


> Calculating SPD metrics for Photo
Finished: 0.025568723678588867
> Calculating KFS metrics for Photo
Finished: 0.02710437774658203
> Calculating layout for Photo
Finished: 0.0030722618103027344
Finish calculating metrics for Photo.


Add Model Inference Results into Writer

In [9]:
## Order: GCN --> MLP --> GCNWUF (20 - 21 - 22)
predicted_classes_GCN, output_vector_GCN = infer_pipeline_2(models_list, "20", root_path, data_path, "Photo")
predicted_classes_MLP, output_vector_MLP = infer_pipeline_2(models_list, "21", root_path, data_path, "Photo")
predicted_classes_GCNWUF, output_vector_GCNWUF = infer_pipeline_2(models_list, "22",  root_path, data_path, "Photo")
writer.add_model("Photo", "GCN", predicted_classes_GCN, output_vector_GCN)
writer.add_model("Photo", "MLP", predicted_classes_MLP, output_vector_MLP)
writer.add_model("Photo", "GCNWUF", predicted_classes_GCNWUF, output_vector_GCNWUF)

0.9115032679738562
0.6369934640522876
0.8483660130718954


Add Explanation Results by Integrated Gradient into Writer

In [10]:
net = GetModel(root_path, models_list, "20")['net']
net.eval()
calculate_subgraph_2(writer, net, data_package_2, features, L, 0, predicted_classes_GCN, "Photo", dgl_graph)
calculate_subgraph_2(writer, net, data_package_2, features, L, 115, predicted_classes_GCN, "Photo", dgl_graph)

Attribution Time:  3.1190104484558105
Attribution Time:  2.9714713096618652


Add Explanation Results by GNNExplainer into Writer

In [11]:
prog_args, explainer = initialize_photo_explainer(net, data_package_2, labels, graph, features, L, idx_train)
for explain_node_idx in [0, 115]:
    print("Explain {}".format(explain_node_idx))
    start_time = time.time()
    GNNExplainer_explain(dgl_graph, explainer, prog_args, explain_node_idx, writer, "Photo", features, 99.99)
    print("Time: {}".format(time.time() - start_time))

(1, 7650, 7650) (1, 7650, 745) (1, 7650) (1, 280) (1, 7650, 8)
Namespace(align_steps=1000, batch_size=20, bias=True, bmname=None, bn=False, ckptdir='ckpt', clean_log=False, clip=2.0, cuda='0', dataset='syn1', dropout=0.0, explain_node=None, explainer_suffix='', gpu=False, graph_idx=-1, graph_mode=False, hidden_dim=20, logdir='log', lr=0.1, mask_act='sigmoid', mask_bias=False, method='base', multigraph_class=-1, multinode_class=-1, name_suffix='', num_epochs=100, num_gc_layers=2, opt='adam', opt_decay_rate=None, opt_decay_step=None, opt_restart=None, opt_scheduler='none', output_dim=20, pkl_fname=None, writer=False)
Explain 0
node label:  6
neigh graph idx:  0 0
Node predicted label:  6
ExplainModule(
  (model): WrapUpGCNModel(
    (net): GCN_hook(
      (gc1): GCN_layer (745 -> 16)
      (gc2): GCN_layer (16 -> 8)
    )
  )
)
finished training in  15.66819715499878
Time: 16.00996446609497
Explain 115
node label:  7
neigh graph idx:  115 31
Node predicted label:  6
ExplainModule(
  (mod

### Flush Writer

In [12]:
writer.flush()